In [ ]:
import math
import random

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import numpy.random as nprand
import pandas as pd
import repsci
import scipy.stats as spstats

from loomio import *
from socialchoice import *
from netdelib import *

%pylab inline
matplotlib.rcParams['font.size'] = 9
matplotlib.rcParams['figure.dpi'] = 150
matplotlib.rcParams['lines.markersize'] = 9

In [ ]:
exp = repsci.Experiment('influence')

In [ ]:
class NetDelibInfluence(object):
    
    def __init__(self, df_score, start=0, end=3):
        
        # Separate control from random-pod
        self.df_control = df_score[df_score.treatment == 1]
        self.df_random = df_score[df_score.treatment == 2]
    
        # Set start/end stage
        self.start = start
        self.end = end
    
        # Create preference sequences
        self.control_preferences = make_preference_sequence_collection(self.df_control)
        self.random_preferences = make_preference_sequence_collection(self.df_random)
        
        # Calculate participant conformity
        self.control_conformity = list(self.conformity_all(self.control_preferences).values())
        self.random_conformity = list(self.conformity_all(self.random_preferences).values())

        # Calculate participant hipness
        self.control_hipness = list(self.hipness_all(self.control_preferences).values())
        self.random_hipness = list(self.hipness_all(self.random_preferences).values())
        

    def pair_conformity(self, p, q):
        '''Quantifies p's conformity toward q.'''
        pi = p[self.start]
        pf = p[self.end]
        qi = q[self.start]
        qf = q[self.end]
        initial = pi.crossing_dissimilarity(qi)
        final = pf.crossing_dissimilarity(qf)
        dp = pi.crossing_dissimilarity(pf)
        dq = qi.crossing_dissimilarity(qf)
        try:
            result = -1 * (final - initial) * (dp / (dq + dp))
        except ZeroDivisionError:
            # Both dq and dp are zero, no contribution to influence of p
            result = 0
        return result
        
        
    def conformity(self, participant_id, preferences):
        p = preferences[participant_id]
        conformity_q = []
        for q_id, q in preferences.items():
            if q_id == participant_id:
                # Avoid self-comparison
                continue
            conformity_q.append(self.pair_conformity(p, q))
        return np.mean(conformity_q)

    def conformity_all(self, preferences):
        results = {}
        for p_id, p in preferences.items():
            results[p_id] = self.conformity(p_id, preferences)
        return results
    
    def hipness(self, participant_id, preferences):
        p = preferences[participant_id]
        hipness_q = []
        for q_id, q in preferences.items():
            if q_id == participant_id:
                # Avoid self-comparison
                continue
            hipness_q.append(self.pair_conformity(q, p))
        return np.mean(hipness_q)
    
    def hipness_all(self, preferences):
        results = {}
        for p_id, p in preferences.items():
            results[p_id] = self.hipness(p_id, preferences)
        return results

    def plot_hipness_hist(self):
        plt.hist([self.control_hipness, self.random_hipness], label=["Control", "Random-Pod"])
        plt.legend()        
    
    def plot_conformity(self, treatment='all', label=None, color=None, linestyle=None):
        if treatment == 'all' or treatment == 'control':
            if label is None:
                l = 'Control'
            else:
                l = label
            x = np.array(sorted(self.control_conformity,reverse=True))
            plt.plot(range(1, len(x)+1), x, label=l, color=color, linestyle=linestyle)
        if treatment == 'all' or treatment == 'random':
            if label is None:
                l = 'Random-Pod'
            else:
                l = label
            x = np.array(sorted(self.random_conformity, reverse=True))
            plt.plot(range(1, len(x)+1), x, label=l, color=color, linestyle=linestyle)
        plt.xlabel('Participant')
        plt.ylabel('Conformity')
        plt.xticks([1, 9, 17, 26, 33])
        plt.ylim([-0.25, 0.25])
        plt.grid()
        plt.legend()
        
    def plot_conformity_cumulative(self):
        x = np.array(sorted(self.control_conformity,reverse=True))
        plt.plot([0] + [sum(x[:i+1]) for i, xi in enumerate(x)], label="Control")
        x = np.array(sorted(self.random_conformity, reverse=True))
        plt.plot([0] + [sum(x[:i+1]) for i, xi in enumerate(x)], label="Random-Pod")
        plt.xlabel('Participant')
        plt.ylabel('Conformity (cumulative)')
        plt.xticks([1, 9, 17, 26, 33])
        plt.ylim([-1.25, 1.25])
        plt.grid()
        plt.legend()
        
    def plot_hipness(self):
        x = np.array(sorted(self.control_hipness,reverse=True))
        plt.plot(range(1, len(x)+1), x, label="Control")
        x = np.array(sorted(self.random_hipness, reverse=True))
        plt.plot(range(1, len(x)+1), x, label="Random-Pod")
        plt.xticks([1, 9, 17, 26, 33])
        plt.ylim([-0.15, 0.15])
        plt.xlabel('Participant')
        plt.ylabel('Hipness')
        plt.grid()
        plt.legend()
        
    def plot_hipness_cumulative(self):
        x = np.array(sorted(self.control_hipness,reverse=True))
        plt.plot([0] + [sum(x[:i+1]) for i, xi in enumerate(x)], label="Control")
        x = np.array(sorted(self.random_hipness, reverse=True))
        plt.plot([0] + [sum(x[:i+1]) for i, xi in enumerate(x)], label="Random-Pod")
        plt.xticks([1, 9, 17, 26, 33])
        plt.ylim([-0.75, 0.75])
        plt.xlabel('Participant')
        plt.ylabel('Hipness (cumulative)')
        plt.grid()
        plt.legend()



In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = fill_attrition(df_score)

netdelib = NetDelibInfluence(df_score)

In [ ]:
plt.figure(figsize=(6,3))

plt.subplot(1,2,1)
netdelib.plot_conformity()

plt.subplot(1,2,2)
netdelib.plot_hipness()

plt.tight_layout()
plt.savefig(exp.get_filename('fig-hipness-conformity.eps'))
plt.savefig(exp.get_filename('fig-hipness-conformity.png'), dpi=600)

In [ ]:
sum(netdelib.control_conformity)

In [ ]:
sum(netdelib.random_conformity)

In [ ]:
def bar_pos_neg(values, bar_width=0.8):
    x = range(len(values))
    sums = []
    
    plt.plot([-0.5, 2.5], [0, 0], 'k-', linewidth=1)
    
    for i in x:
        sums.append(sum(values[i]))
        positive = sorted([v for v in values[i] if v > 0], reverse=True)
        negative = sorted([v for v in values[i] if v < 0])

        y = 0
        for val in positive:
            ax.add_patch(Rectangle((i - bar_width / 2, y), bar_width, val, facecolor='#b6b', edgecolor='#848'))
            y += val

        y = 0
        for val in negative:
            ax.add_patch(Rectangle((i - bar_width / 2, y), bar_width, val, facecolor='#bb6', edgecolor='#884'))
            y += val

    plt.plot(x, sums, '.-k')
            
plt.figure(figsize=(8,8))

ax = plt.subplot(2,2,1)
plt.title('Control')
values = [
    NetDelibInfluence(df_score, start=0, end=(i+1)).control_conformity
    for i in range(3)]
bar_pos_neg(values)
plt.xlim([-0.5, 2.5])
plt.ylim([-1.6, 1.6])
plt.xticks(range(3), labels=range(1,4))
plt.xlabel('Round')
plt.ylabel('Participant Conformity')

ax = plt.subplot(2,2,2)
plt.title('Random-Pod')
values = [
    NetDelibInfluence(df_score, start=0, end=(i+1)).random_conformity
    for i in range(3)]
bar_pos_neg(values)
plt.xlim([-0.5, 2.5])
plt.ylim([-1.6, 1.6])
plt.xticks(range(3), labels=range(1,4))
plt.xlabel('Round')
plt.ylabel('Participant Conformity')

ax = plt.subplot(2,2,3)
plt.title('Control')
values = [
    NetDelibInfluence(df_score, start=0, end=(i+1)).control_hipness
    for i in range(3)]
bar_pos_neg(values)
plt.xlim([-0.5, 2.5])
plt.ylim([-1.6, 1.6])
plt.xticks(range(3), labels=range(1,4))
plt.xlabel('Round')
plt.ylabel('Participant Hipness')

ax = plt.subplot(2,2,4)
plt.title('Random-Pod')
values = [
    NetDelibInfluence(df_score, start=0, end=(i+1)).random_hipness
    for i in range(3)]
bar_pos_neg(values)
plt.xlim([-0.5, 2.5])
plt.ylim([-1.6, 1.6])
plt.xticks(range(3), labels=range(1,4))
plt.xlabel('Round')
plt.ylabel('Participant Hipness')

plt.tight_layout()
plt.savefig(exp.get_filename('fig-influence-bar.eps'))
plt.savefig(exp.get_filename('fig-influence-bar.png'), dpi=600)
